In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt  #导入作图包
import os
import numpy as np
import pylab
import glob

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.chdir("/home/wxming/PycharmProject/Notebook/Concrete-Crack-Detection/")  #改变改变工作目录到此目录
from raw_to_tfrecoder import slide

# modir = "/home/wxming/PycharmProject/Notebook/Concrete-Crack-Detection/log/model/model.cpk-89000"
modir = "/home/wxming/PycharmProject/Notebook/Concrete-Crack-Detection/log/new/model.cpk-11000"
imaga_dir = "/home/wxming/PycharmProject/Notebook/Concrete-Crack-Detection/data"


/home/wxming/PycharmProject/environ/All/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
pylab.rcParams['figure.figsize'] = (25.0, 8.0)
image_list = glob.glob(imaga_dir + "/*.jpg")
# print(image_list)

with tf.Graph().as_default() as g1:
    with tf.Session() as sess:
        tf.train.import_meta_graph(modir + ".meta")
        restore = tf.train.Saver()
        restore.restore(sess, modir)
        input = g1.get_tensor_by_name("IteratorGetNext:0")
        output = g1.get_tensor_by_name("output/flatten/Reshape:0")
        prey = tf.argmax(output, 1)
        # sess.run(tf.global_variables_initializer())

        for im in image_list:
            image_list = []
            data = slide(im, 114)
            x, iterator = data.break_image()
            sess.run(iterator)
            while True:
                try:
                    image, h, w = sess.run([x['image'], x['h'], x['w']])
                    prey_val = sess.run(prey, feed_dict={input: image})
                    image_list.extend(prey_val)
                except:
                    break
            a = np.array(image_list).reshape([h[0], w[0]])
            pr_image, raw_image = data.marge_image(a)
            ax = plt.figure()
            print("=========================================", im.split("/")[-1],
                  "=========================================")
            ax.add_subplot(1, 2, 1).imshow(raw_image)
            ax.add_subplot(1, 2, 2).imshow(pr_image)
            plt.show()
sess.close()
